In [1]:
import pybullet as p
import time
import os
import math
import pybullet_data
import numpy as np
physicsClient = p.connect(p.GUI)
p.configureDebugVisualizer(p.COV_ENABLE_Y_AXIS_UP, 1)
p.setGravity(0,-9.8,0)

# load URDFs 

In [2]:
p.setAdditionalSearchPath(pybullet_data.getDataPath()) 

StartPos = [2,2,1]
flipconst = p.getQuaternionFromEuler([-math.pi * 0.5, 0, 0])
planeId = p.loadURDF("plane.urdf", [0,0,0],flipconst,useMaximalCoordinates=True)
humId = p.loadURDF("./urdfs/humanoid.urdf",
                   basePosition=StartPos, 
                   baseOrientation=p.getQuaternionFromEuler([-math.pi,0,math.pi]),
                  globalScaling=0.25,
                    useFixedBase=False)

In [3]:
for joint in range(p.getNumJoints(humId)):
    p.changeDynamics(humId, joint, lateralFriction=0.9)

p.changeDynamics(humId, -1, linearDamping=0, angularDamping=0)

chest = 1
neck = 2
rightHip = 3
rightKnee = 4
rightAnkle = 5
rightShoulder = 6
rightElbow = 7
leftHip = 9
leftKnee = 10
leftAnkle = 11
leftShoulder = 12
leftElbow = 13

#following deep mimic example..
jointIndicesWhole = [
    chest, neck, rightHip, rightKnee, rightAnkle, rightShoulder, rightElbow, leftHip, leftKnee,
    leftAnkle, leftShoulder, leftElbow
]
jointFrictionForce =0



for joint in jointIndicesWhole:
    # self._pybullet_client.setJointMotorControlMultiDof(self._sim_model, j, self._pybullet_client.POSITION_CONTROL, force=[1,1,1])
    p.setJointMotorControl2(humId, #bodyUniqueID
                          joint, #jointIndex
                          p.POSITION_CONTROL, #ControlMode 
                          # you specify a target position for the joint, and the motor control will attempt to move the joint to that target position  
                          # It will apply ******(forces or torques as needed)매우중요! to achieve the desired joint position. 
                          targetPosition=0,
                          positionGain=0,
                          targetVelocity=0,
                          force=jointFrictionForce) #(Myi) jointFrictionForce set to 0 in the above
    p.setJointMotorControlMultiDof(
          humId,
          joint,
          p.POSITION_CONTROL,
          targetPosition=[0, 0, 0, 1],
          targetVelocity=[0, 0, 0],
          positionGain=0,
          velocityGain=1,
          force=[jointFrictionForce, jointFrictionForce, jointFrictionForce])

jointDofNum = [4, 4, 4, 1, 4, 4, 1, 4, 1, 4, 4, 1]

def setForce(taus):
    indices, forces = [] , []
    dofIndex = 7
    for joint in range(len(jointIndicesWhole)):
        # print(indices)
        jtindx = jointIndicesWhole[joint]
        indices.append(jtindx)   
    
        if jointDofNum[joint] == 4:
            force = [
              1 * taus[dofIndex + 0], 1 * taus[dofIndex + 1],
              1 * taus[dofIndex + 2]
            ]
        if jointDofNum[joint] == 1:
            force = [1 * taus[dofIndex]]
            #print("force[", jointIndex,"]=",force)
        forces.append(force)
        dofIndex += jointDofNum[joint]
        
    p.setJointMotorControlMultiDofArray(humId,
                                         indices,
                                         p.TORQUE_CONTROL,
                                         forces=forces)

In [4]:
for i in range (2000):
    # print("step ", i)
    setForce(np.zeros(100).tolist())
    p.stepSimulation()
    time.sleep(1./240.)
    # print("position/orientation : ", p.getBasePositionAndOrientation(humId))

error: Not connected to physics server.

In [ ]:
help(p.stepSimulation())

In [ ]:
help(p.loadURDF)